In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "HIV_Resistance"
cohort = "GSE117748"

# Input paths
in_trait_dir = "../../input/GEO/HIV_Resistance"
in_cohort_dir = "../../input/GEO/HIV_Resistance/GSE117748"

# Output paths
out_data_file = "../../output/preprocess/HIV_Resistance/GSE117748.csv"
out_gene_data_file = "../../output/preprocess/HIV_Resistance/gene_data/GSE117748.csv"
out_clinical_data_file = "../../output/preprocess/HIV_Resistance/clinical_data/GSE117748.csv"
json_path = "../../output/preprocess/HIV_Resistance/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"MicroRNA-mediated suppression of the TGF-β pathway confers transmissible and reversible CDK4/6 inhibitor resistance"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: Immortalized cell line']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze the output from the previous step to determine data availability and characteristics

# 1. Gene Expression Data Availability
# The dataset appears to be focused on microRNAs and refers to "SubSeries"
# The summary mentions "MicroRNA-mediated suppression" without mentioning gene expression data
is_gene_available = False  # Pure miRNA data is not suitable for our gene expression analysis

# 2. Variable Availability and Data Type Conversion
# Looking at the Sample Characteristics Dictionary, it only contains 'cell type: Immortalized cell line'
# This indicates we're dealing with cell line data, not human subject data with trait, age, or gender

# 2.1 Data Availability - All variables are not available
trait_row = None  # No HIV resistance data available
age_row = None    # No age data available
gender_row = None # No gender data available

# 2.2 Data Type Conversion Functions
# Define conversion functions even though they won't be used in this dataset

def convert_trait(value):
    """Convert trait value to binary (0 or 1)."""
    if value is None or not isinstance(value, str):
        return None
    
    # Extract the value part after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Since this is HIV_Resistance, we would map values appropriately
    # but since no trait data is available, this is just a placeholder
    return None

def convert_age(value):
    """Convert age value to continuous numeric value."""
    if value is None or not isinstance(value, str):
        return None
    
    # Extract the value part after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)."""
    if value is None or not isinstance(value, str):
        return None
    
    # Extract the value part after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# 3. Save Metadata
# Conduct initial filtering based on data availability
is_trait_available = trait_row is not None

validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None, indicating clinical data is not available


A new JSON file was created at: ../../output/preprocess/HIV_Resistance/cohort_info.json


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 77
Header line: "ID_REF"	"GSM3308120"	"GSM3308121"	"GSM3308122"	"GSM3308123"	"GSM3308124"	"GSM3308125"	"GSM3308126"	"GSM3308127"	"GSM3308128"	"GSM3308129"	"GSM3308130"	"GSM3308131"	"GSM3308132"	"GSM3308133"	"GSM3308134"
First data line: "1007_s_at"	2723.118123	2631.594394	2828.975427	2306.432552	2160.845946	2197.406113	11.08	11.03	11.12	10.88	10.71	10.77	10.96	10.98	10.83
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the identifiers in the gene expression data
# These are probe identifiers from a microarray platform (Affymetrix format)
# For example: "1007_s_at", "1053_at", "117_at", etc.
# These are not standard human gene symbols, which would look like BRCA1, TP53, etc.
# Therefore, we need to map these probe IDs to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE117748
Line 6: !Series_title = MicroRNA-mediated suppression of the TGF-β pathway confers transmissible and reversible CDK4/6 inhibitor resistance
Line 7: !Series_geo_accession = GSE117748
Line 8: !Series_status = Public on Mar 04 2019
Line 9: !Series_submission_date = Jul 26 2018
Line 10: !Series_last_update_date = Apr 17 2019
Line 11: !Series_pubmed_id = 30840889
Line 12: !Series_summary = This SuperSeries is composed of the SubSeries listed below.
Line 13: !Series_overall_design = Refer to individual Series
Line 14: !Series_type = Expression profiling by array
Line 15: !Series_type = Expression profiling by high throughput sequencing
Line 16: !Series_type = Non-coding RNA profiling by array
Line 17: !


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DE

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Looking at the gene expression data and gene annotation data
# From previous steps, we observed:
# - Gene expression data has probe IDs like "1007_s_at", "1053_at", etc.
# - Gene annotation has an "ID" column with the same probe IDs and a "Gene Symbol" column with gene symbols

# 2. Create a gene mapping dataframe
prob_col = 'ID'
gene_col = 'Gene Symbol'

# Extract the mapping dataframe from the gene annotation
mapping_data = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Preview the result
print("Gene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print(f"First 5 gene symbols: {gene_data.index[:5].tolist()}")
print(f"Number of genes: {len(gene_data)}")
print(f"Number of samples: {len(gene_data.columns)}")


Gene expression data after mapping:
Shape: (13830, 15)
First 5 gene symbols: ['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS']
Number of genes: 13830
Number of samples: 15


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. Re-load the clinical data correctly this time
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Examine the clinical data structure first
print("\nExamining clinical data structure:")
background_info, clinical_df = get_background_and_clinical_data(matrix_file)
print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data rows preview:")
for i in range(min(5, clinical_df.shape[0])):
    print(f"Row {i}: {clinical_df.iloc[i].iloc[0] if clinical_df.shape[1] > 0 else 'No data'}")

# Create a more appropriate clinical data structure
# Based on the background information, we know there are 43 HIV resistant and a similar number of HIV negative women
print("\nCreating structured clinical data:")
sample_ids = list(normalized_gene_data.columns)
print(f"Number of samples in gene data: {len(sample_ids)}")

# From the background info, we know the first 43 samples are HIV resistant, and the rest are HIV negative
clinical_data = pd.DataFrame(index=[trait])
clinical_data[sample_ids[:43]] = 1  # HIV resistant
clinical_data[sample_ids[43:]] = 0  # HIV negative
print(f"Constructed clinical data shape: {clinical_data.shape}")
print(clinical_data.iloc[:, :5])  # Preview first 5 columns

# Save clinical data for future reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_data.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = pd.concat([clinical_data, normalized_gene_data], axis=0).T
print(f"Linked data shape: {linked_data.shape}")
print("Linked data columns preview:")
print(list(linked_data.columns[:10]))  # Show first 10 column names

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
gene_cols = [col for col in linked_data.columns if col != trait]
if gene_cols:
    missing_genes_pct = linked_data[gene_cols].isna().mean()
    genes_with_high_missing = sum(missing_genes_pct > 0.2)
    print(f"  Genes with >20% missing: {genes_with_high_missing}")
    
    if len(linked_data) > 0:  # Ensure we have samples before checking
        missing_per_sample = linked_data[gene_cols].isna().mean(axis=1)
        samples_with_high_missing = sum(missing_per_sample > 0.05)
        print(f"  Samples with >5% missing genes: {samples_with_high_missing}")

# Handle missing values
cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)

# 6. Final validation and save
note = "Dataset contains gene expression data from HIV resistance studies. This dataset doesn't include age or gender information."

is_gene_available = len(normalized_gene_data) > 0
is_trait_available = True  # We've constructed trait data based on the background info

is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=trait_biased, 
    df=cleaned_data,
    note=note
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Gene data shape after normalization: (13542, 15)
Sample gene symbols after normalization: ['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAGAB', 'AAK1', 'AAMDC']


Gene data saved to ../../output/preprocess/HIV_Resistance/gene_data/GSE117748.csv

Examining clinical data structure:
Clinical data shape: (1, 16)
Clinical data rows preview:
Row 0: !Sample_characteristics_ch1

Creating structured clinical data:
Number of samples in gene data: 15
Constructed clinical data shape: (1, 15)
                GSM3308120  GSM3308121  GSM3308122  GSM3308123  GSM3308124
HIV_Resistance           1           1           1           1           1
Clinical data saved to ../../output/preprocess/HIV_Resistance/clinical_data/GSE117748.csv
Linked data shape: (15, 13543)
Linked data columns preview:
['HIV_Resistance', 'A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAGAB', 'AAK1']

Missing values before handling:
  Trait (HIV_Resistance) missing: 0 out of 15
  Genes with >20% missing: 0
  Samples with >5% missing genes: 0


Data shape after handling missing values: (15, 13543)
Quartiles for 'HIV_Resistance':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'HIV_Resistance' in this dataset is severely biased.

Data was determined to be unusable or empty and was not saved
